<h1 style="text-align: center;">RNN MODEL WITH MACD TRIGGER</h1>

<h2>Initialization</h2>

In [1]:
# Libraries
import MetaTrader5 as mt5  # pip install MetaTrader5
import pandas as pd  # pip install pandas
import talib
import numpy as np
from datetime import datetime
import plotly.express as px  # pip install plotly

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

c:\Users\jeanc\trading\rnn\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# start the platform with initialize()
mt5.initialize()

True

In [3]:
# login to Trade Account with login()
# make sure that trade server is enabled in MT5 client terminal

login = 5044174558
password = '-a6bNxSy'
server = 'MetaQuotes-Demo'

mt5.login(login, password, server)

True

In [4]:
# get account info
account_info = mt5.account_info()
print(account_info)

# getting specific account data
login_number = account_info.login
balance = account_info.balance
equity = account_info.equity

print()
print('login: ', login_number)
print('balance: ', balance)
print('equity: ', equity)

AccountInfo(login=5044174558, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=1000.0, credit=0.0, profit=0.0, equity=1000.0, margin=0.0, margin_free=1000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jean-Charles Jacques', server='MetaQuotes-Demo', currency='EUR', company='MetaQuotes Ltd.')

login:  5044174558
balance:  1000.0
equity:  1000.0


<h2>GET MARKET DATA</h2>

In [5]:
symbol = "EURUSD"
info = mt5.symbol_info(symbol)

point = info.point       # ex: 0.00001
digits = info.digits     # ex: 5

pip_value = point * 10   # 1 pip en valeur de prix


In [ ]:
# ohlc_data
ohlc_data = pd.DataFrame(mt5.copy_rates_range(symbol, 
                                             mt5.TIMEFRAME_D1, 
                                             datetime(2021, 1, 1), 
                                             datetime.now()))

fig = px.line(ohlc_data, x=ohlc_data['time'], y=ohlc_data['close'])
fig.show()

ohlc_data

In [ ]:
ohlc_data.to_csv('data/eurusd.csv')

<h2>COMMON FUNCTIONS</h2>

In [6]:
def plot_loss(history, title="Model Loss Over Epochs"):
    loss_df = pd.DataFrame({
        "Epoch": range(1, len(history.history["loss"]) + 1),
        "Train Loss": history.history["loss"],
        "Validation Loss": history.history.get("val_loss")
    })

    fig = px.line(
        loss_df,
        x="Epoch",
        y=["Train Loss", "Validation Loss"],
        title=title,
        labels={"value": "Loss", "variable": "Metric"}
    )

    fig.update_layout(
        hovermode="x unified",
        template="plotly_white"
    )

    fig.show()


<h2>TRAIN MODEL</h2>

In [7]:
# Charger le dataset
df = pd.read_csv("data/eurusd.csv")
df = df[['close', 'tick_volume']].dropna()

In [8]:
# Calcul du MACD
macd, macd_signal, macd_hist = talib.MACD(
    df["close"].values,
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

df["macd"] = macd
df["macd_signal"] = macd_signal
df["macd_hist"] = macd_hist

In [9]:
# Calcul du trigger
df["macd_trigger"] = 0

df.loc[
    (df["macd_hist"].shift(1) < 0) & (df["macd_hist"] > 0),
    "macd_trigger"
] = 1   # LONG

df.loc[
    (df["macd_hist"].shift(1) > 0) & (df["macd_hist"] < 0),
    "macd_trigger"
] = -1  # SHORT


In [10]:
features = ["close", "tick_volume", "macd_hist"]
df_model = df[features + ["macd_trigger"]].dropna()

In [11]:
# Normalisation (CRUCIAL pour un RNN)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_model[features])

In [12]:
WINDOW_SIZE = 60
HORIZON = 1

def create_sequences_on_trigger(df, scaled_data, window, horizon):
    X, y, idx_list = [], [], []

    for i in range(window, len(df) - horizon):
        if df["macd_trigger"].iloc[i] == 0:
            continue

        X.append(scaled_data[i-window:i])

        delta = df["close"].iloc[i + horizon] - df["close"].iloc[i]
        y.append(np.sign(delta))

        idx_list.append(df.index[i])  # 🔥 timestamp exact de l'exemple

    return np.array(X), np.array(y), np.array(idx_list)



In [13]:
X, y, idx = create_sequences_on_trigger(df_model, scaled_features, WINDOW_SIZE, HORIZON)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
idx_train, idx_test = idx[:split], idx[split:]


In [14]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(WINDOW_SIZE, X.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dense(1, activation="tanh")  # sortie ∈ [-1, +1]
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=[]
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=40,
    batch_size=16,
    verbose=1
)


Epoch 1/40


c:\Users\jeanc\trading\rnn\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - loss: 1.0166 - val_loss: 0.9325
Epoch 2/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.0031 - val_loss: 0.9623
Epoch 3/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.9916 - val_loss: 0.9706
Epoch 4/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.9993 - val_loss: 1.0192
Epoch 5/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.9949 - val_loss: 1.0226
Epoch 6/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.9826 - val_loss: 0.9906
Epoch 7/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.9864 - val_loss: 0.9653
Epoch 8/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.9982 - val_loss: 0.9655
Epoch 9/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.9814 - val_loss: 1.0050
Epoch 10/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.9807 - val_loss: 1.0448
Epoch 11/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.9836 - val_loss: 1.0349
Epoch 12/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9738 - val_loss: 0.9916
Epoch 13/40


In [15]:
pred = model.predict(X_test).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


<h2>TEST VALIDATION</h2>

In [16]:
def build_trade_df(
    df_model,
    df_full,
    y_part,
    pred_part,
    idx_part,
    horizon
):
    trade_df = pd.DataFrame(index=idx_part)

    trade_df["close"] = df_full.loc[idx_part, "close"].values
    trade_df["macd"] = df_full.loc[idx_part, "macd"].values
    trade_df["macd_trigger"] = df_model.loc[idx_part, "macd_trigger"].values
    trade_df["model_pred"] = pred_part
    trade_df["real_dir"] = y_part

    # 🔥 règle MACD + LSTM directionnelle
    trade_df["trade"] = 0

    trade_df.loc[
        (trade_df["macd_trigger"] == 1) &
        (trade_df["macd"] > 0) &
        (trade_df["model_pred"] > 0),
        "trade"
    ] = 1

    trade_df.loc[
        (trade_df["macd_trigger"] == -1) &
        (trade_df["macd"] < 0) &
        (trade_df["model_pred"] < 0),
        "trade"
    ] = -1

    trade_df["future_close"] = df_full["close"].shift(-horizon).loc[idx_part]
    trade_df["real_delta"] = trade_df["future_close"] - trade_df["close"]
    trade_df["trade_result"] = np.sign(trade_df["real_delta"])

    trade_df["hit"] = (trade_df["trade"] == trade_df["trade_result"]).astype(int)

    trade_df = trade_df[trade_df["trade"] != 0]

    return trade_df.dropna()


In [17]:
def trading_accuracy(trade_df):
    traded = trade_df["trade"] != 0
    n_trades = traded.sum()
    n_total = len(trade_df)

    if n_trades == 0:
        return {
            "Trades": 0,
            "Trade_frequency_%": 0.0,
            "Accuracy_%": np.nan
        }

    accuracy = trade_df.loc[traded, "hit"].mean() * 100

    return {
        "Trades": int(n_trades),
        "Trade_frequency_%": n_trades / n_total * 100,
        "Accuracy_%": accuracy
    }


In [18]:
import plotly.graph_objects as go

def plot_trades(trade_df, df_full, title="MACD + LSTM Trades (D1)"):
    fig = go.Figure()

    # prix
    fig.add_trace(go.Scatter(
        x=df_full.index,
        y=df_full["close"],
        mode="lines",
        name="Close",
        line=dict(color="black")
    ))

    # LONG
    long_trades = trade_df[trade_df["trade"] == 1]
    fig.add_trace(go.Scatter(
        x=long_trades.index,
        y=long_trades["close"],
        mode="markers",
        name="LONG",
        marker=dict(symbol="triangle-up", size=10, color="green")
    ))

    # SHORT
    short_trades = trade_df[trade_df["trade"] == -1]
    fig.add_trace(go.Scatter(
        x=short_trades.index,
        y=short_trades["close"],
        mode="markers",
        name="SHORT",
        marker=dict(symbol="triangle-down", size=10, color="red")
    ))

    fig.update_layout(
        title=title,
        hovermode="x unified",
        template="plotly_white"
    )

    fig.show()


In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_trades_with_macd(
    trade_df,
    df_full,
    title="MACD + LSTM Trades (D1)"
):
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        row_heights=[0.65, 0.35],
        subplot_titles=("Price & Trades", "MACD")
    )

    # =====================
    # 1️⃣ PRIX
    # =====================
    fig.add_trace(
        go.Scatter(
            x=df_full.index,
            y=df_full["close"],
            mode="lines",
            name="Close",
            line=dict(color="black", width=1)
        ),
        row=1, col=1
    )

    # LONG
    long_trades = trade_df[trade_df["trade"] == 1]
    fig.add_trace(
        go.Scatter(
            x=long_trades.index,
            y=long_trades["close"],
            mode="markers",
            name="LONG",
            marker=dict(symbol="triangle-up", size=10, color="green")
        ),
        row=1, col=1
    )

    # SHORT
    short_trades = trade_df[trade_df["trade"] == -1]
    fig.add_trace(
        go.Scatter(
            x=short_trades.index,
            y=short_trades["close"],
            mode="markers",
            name="SHORT",
            marker=dict(symbol="triangle-down", size=10, color="red")
        ),
        row=1, col=1
    )

    # =====================
    # 2️⃣ MACD
    # =====================
    fig.add_trace(
        go.Scatter(
            x=df_full.index,
            y=df_full["macd"],
            mode="lines",
            name="MACD",
            line=dict(color="blue")
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_full.index,
            y=df_full["macd_signal"],
            mode="lines",
            name="Signal",
            line=dict(color="orange", dash="dot")
        ),
        row=2, col=1
    )

    # Histogramme MACD
    fig.add_trace(
        go.Bar(
            x=df_full.index,
            y=df_full["macd_hist"],
            name="Histogram",
            marker_color=df_full["macd_hist"].apply(
                lambda x: "green" if x >= 0 else "red"
            ),
            opacity=0.4
        ),
        row=2, col=1
    )

    # =====================
    # LAYOUT
    # =====================
    fig.update_layout(
        title=title,
        hovermode="x unified",
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    fig.show()


In [20]:
def direction_metrics(trade_df):
    """
    trade_df doit contenir :
    - trade        ∈ {-1, +1}
    - trade_result ∈ {-1, +1}
    - hit          ∈ {0, 1}
    """

    metrics = {}

    n_trades = len(trade_df)
    n_ok = trade_df["hit"].sum()
    n_nok = n_trades - n_ok

    metrics["Trades"] = int(n_trades)
    metrics["Direction_OK"] = int(n_ok)
    metrics["Direction_NOK"] = int(n_nok)
    metrics["Accuracy_%"] = float(100 * n_ok / n_trades) if n_trades > 0 else np.nan

    # --- LONG / SHORT breakdown ---
    long_trades = trade_df[trade_df["trade"] == 1]
    short_trades = trade_df[trade_df["trade"] == -1]

    if len(long_trades) > 0:
        metrics["LONG_trades"] = int(len(long_trades))
        metrics["LONG_accuracy_%"] = float(100 * long_trades["hit"].mean())
    else:
        metrics["LONG_trades"] = 0
        metrics["LONG_accuracy_%"] = np.nan

    if len(short_trades) > 0:
        metrics["SHORT_trades"] = int(len(short_trades))
        metrics["SHORT_accuracy_%"] = float(100 * short_trades["hit"].mean())
    else:
        metrics["SHORT_trades"] = 0
        metrics["SHORT_accuracy_%"] = np.nan

    # --- biais directionnel ---
    metrics["LONG_%"] = float(100 * (trade_df["trade"] == 1).mean())
    metrics["SHORT_%"] = float(100 * (trade_df["trade"] == -1).mean())

    return metrics



In [21]:
pred_test = model.predict(X_test).flatten()

trade_df_test = build_trade_df(
    df_model=df_model,
    df_full=df,
    y_part=y_test,
    pred_part=pred_test,
    idx_part=idx_test,
    horizon=HORIZON
)

metrics_test = direction_metrics(trade_df_test)

print("📊 Directional metrics (TEST – MACD filtered):")
for k, v in metrics_test.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2f}")
    else:
        print(f"{k}: {v}")

plot_trades_with_macd(trade_df_test, df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
📊 Directional metrics (TEST – MACD filtered):
Trades: 4
Direction_OK: 2
Direction_NOK: 2
Accuracy_%: 50.00
LONG_trades: 2
LONG_accuracy_%: 50.00
SHORT_trades: 2
SHORT_accuracy_%: 50.00
LONG_%: 50.00
SHORT_%: 50.00
